In [3]:
import pandas as pd
import requests, warnings
import seasonal_plot
from datetime import datetime as dt

In [4]:
def ag_get_data(query):
    warnings.simplefilter('ignore', requests.packages.urllib3.exceptions.InsecureRequestWarning)
    params = {"Database": "OIL", "Query": query}
    resp = requests.post('https://TST-QDEV-AP1.petroineos.local:5001/genericdata/Fetch/', json=params, verify=False)
    json_result = resp.json()
    df = pd.DataFrame(json_result)
    return df

In [5]:
sched_qry = """select ddate, Unit_ID, Capacity, SUB_REGION from oil.Refinery.RefineryUnitCapacityMonthly
where UNIT_GROUP='CDU' and COUNTRY='France' and DDate>'2022-08-01' and DDate<'2022-10-01'"""

In [6]:
outage_qry = """select RU.COUNTRY,lst.PLANT_ID, d.UNIT_ID, d.EVENT_ID,d.EVENT_PDATE, d.OUTAGE_DATE, d.CAP_OFFLINE from oil.Refinery.MaintenanceDaily d
left join oil.Refinery.RefineryMaintenanceBestLatest lst on d.EVENT_ID=lst.EVENT_ID
left join oil.Refinery.RefineryUnit RU on RU.UNIT_ID=d.UNIT_ID
where d.PDate = (select max(pdate) from oil.Refinery.MaintenanceDaily) and RU.COUNTRY = 'France' and d.OUTAGE_DATE>'2022-08-01' and d.OUTAGE_DATE<'2022-11-01'
"""

In [7]:
outages = ag_get_data(outage_qry)

In [8]:
cap = ag_get_data(sched_qry)

In [10]:
cap['ddate'] = pd.to_datetime(cap['ddate'])

In [11]:
cap.set_index('ddate', inplace=True)

In [15]:
cap

Unit_ID  Capacity SUB_REGION
ddate                                   
2022-09-01  1060406   70000.0        NWE
2022-09-01  1060407  170000.0        NWE
2022-09-01  1063667  140000.0        Med
2022-09-01  1064037       0.0        NWE
2022-09-01  1064622  230000.0        NWE
2022-09-01  1065505  117000.0        Med
2022-09-01  1066443  218000.0        Med
2022-09-01  1066618  205600.0        NWE

In [14]:
cap.resample('d').mean()

Capacity
ddate               
2022-09-01  143825.0